## Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
from urllib.request import urlopen
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

print('Libraries imported.')

Libraries imported.


## Fetch Location Coordinates

In [2]:
address = 'New York, New York'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate are {}, {}.'.format(latitude, longitude))

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate are 40.7127281, -74.0060152.


## Configure Foursquare API Settings

In [3]:
CLIENT_ID = '2G0MKVHCH3TME4LE2WXMB05RNA53M3HEO020NOR5YYSZD4VU' # your Foursquare ID
CLIENT_SECRET = 'MVVTOEEO4SRQ1XBNX30DYQKQAUJDPO3DRK0LBBKHSR5JLUUF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2G0MKVHCH3TME4LE2WXMB05RNA53M3HEO020NOR5YYSZD4VU
CLIENT_SECRET:MVVTOEEO4SRQ1XBNX30DYQKQAUJDPO3DRK0LBBKHSR5JLUUF


In [4]:
LIMIT = 300 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=2G0MKVHCH3TME4LE2WXMB05RNA53M3HEO020NOR5YYSZD4VU&client_secret=MVVTOEEO4SRQ1XBNX30DYQKQAUJDPO3DRK0LBBKHSR5JLUUF&v=20180605&ll=40.7127281,-74.0060152&radius=500&limit=300'

## Fetched JSON Data

In [5]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5cf7c5cf4c1f6753b644d37d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Downtown Manhattan',
  'headerFullLocation': 'Downtown Manhattan, New York',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 177,
  'suggestedBounds': {'ne': {'lat': 40.7172281045, 'lng': -74.00008952063419},
   'sw': {'lat': 40.7082280955, 'lng': -74.0119408793658}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '57f0689d498e7d49d9189369',
       'name': 'The Bar Room at Temple Court',
       'location': {'address': '123 Nassau St',
        'lat': 40.7114477287544,
        'lng': -74.00680157032005,
        'labe

In [6]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
filtered_columns = ['venue.name', 'venue.id', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues

,venue.name,venue.id,venue.categories,venue.location.lat,venue.location.lng
0,The Bar Room at Temple Court,57f0689d498e7d49d9189369,Hotel Bar,40.711448,-74.006802
1,The Beekman - A Thompson Hotel,56d8c0f8498edb854f926e6a,Hotel,40.711173,-74.006702
2,City Hall Park,3fd66200f964a520d8f11ee3,Park,40.712241,-74.006977
3,Alba Dry Cleaner & Tailor,4c606c3e1e5cd13ad1a1a1ed,Laundry Service,40.711434,-74.006272
4,The Wooly Daily,56093809498e5344ab8835a6,Coffee Shop,40.712137,-74.008395
5,Gibney Dance Center Downtown,53373f26498e940581c90985,Dance Studio,40.713923,-74.005661
6,Augustine,58191674ded8f8626ed70af0,French Restaurant,40.711310,-74.006660
7,The Class by Taryn Toomey,58a253c830ecc66c9e5b40a0,Gym / Fitness Center,40.712753,-74.008734
8,Takahachi Bakery,4c154c9a77cea593c401d260,Bakery,40.713653,-74.008804
9,Aahar Indian Cuisine,575dea4c498e2739e43a27e2,Indian Restaurant,40.713307,-74.007994


## Column Features are named properly

In [11]:
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,id,categories,lat,lng
0,The Bar Room at Temple Court,57f0689d498e7d49d9189369,Hotel Bar,40.711448,-74.006802
1,The Beekman - A Thompson Hotel,56d8c0f8498edb854f926e6a,Hotel,40.711173,-74.006702
2,City Hall Park,3fd66200f964a520d8f11ee3,Park,40.712241,-74.006977
3,Alba Dry Cleaner & Tailor,4c606c3e1e5cd13ad1a1a1ed,Laundry Service,40.711434,-74.006272
4,The Wooly Daily,56093809498e5344ab8835a6,Coffee Shop,40.712137,-74.008395
5,Gibney Dance Center Downtown,53373f26498e940581c90985,Dance Studio,40.713923,-74.005661
6,Augustine,58191674ded8f8626ed70af0,French Restaurant,40.711310,-74.006660
7,The Class by Taryn Toomey,58a253c830ecc66c9e5b40a0,Gym / Fitness Center,40.712753,-74.008734
8,Takahachi Bakery,4c154c9a77cea593c401d260,Bakery,40.713653,-74.008804
9,Aahar Indian Cuisine,575dea4c498e2739e43a27e2,Indian Restaurant,40.713307,-74.007994


In [12]:
nearby_venues['categories'].unique()

array(['Hotel Bar', 'Hotel', 'Park', 'Laundry Service', 'Coffee Shop',
       'Dance Studio', 'French Restaurant', 'Gym / Fitness Center',
       'Bakery', 'Indian Restaurant', 'Falafel Restaurant', 'Gym',
       'Sandwich Place', 'Yoga Studio', 'Monument / Landmark',
       'American Restaurant', 'Boxing Gym', 'Cuban Restaurant',
       'Sporting Goods Shop', 'Building', 'Liquor Store', 'Pizza Place',
       'Bar', 'Plaza', 'Event Space', 'Burger Joint',
       'Furniture / Home Store', 'Italian Restaurant', 'Spa',
       'Middle Eastern Restaurant', 'Antique Shop',
       'Fast Food Restaurant', 'Burrito Place', 'Juice Bar',
       'Cosmetics Shop', 'Art Gallery', "Women's Store", 'Strip Club',
       'Japanese Restaurant', 'Speakeasy', 'Baby Store', 'Bookstore',
       'Wine Bar', 'Restaurant', 'Coworking Space',
       'Health & Beauty Service', 'Café', 'Discount Store',
       'Greek Restaurant', 'Cocktail Bar', 'Nail Salon', 'Poke Place',
       'Kids Store', 'Molecular Gastronom

In [13]:
nearby_venues.shape

(100, 5)